In [0]:
%sql
SHOW VOLUMES;
SHOW CATALOGS;

SHOW SCHEMAS IN workspace;


databaseName
default
information_schema


In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, split, size, year, try_to_date, when, round, avg, count

spark = SparkSession.builder.appName("MoviesDataPipeline").getOrCreate()

# Full path to your volume
full_volume_path = "/Volumes/workspace/default/movies_data/"

# Use DROPMALFORMED mode to ignore broken rows
df_movies = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("mode", "DROPMALFORMED")
    .csv(f"{full_volume_path}movies_metadata.csv")
)

df_credits = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("mode", "DROPMALFORMED")
    .csv(f"{full_volume_path}credits.csv")
)

df_keywords = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("mode", "DROPMALFORMED")
    .csv(f"{full_volume_path}keywords.csv")
)

# Preview
display(df_movies.limit(5))



adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,null,Toy Story,False,7.7,5415
False,null,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",null,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",null,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",null,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of Hi

In [0]:
from pyspark.sql.functions import year, to_date, size

# Safely cast numeric columns
numeric_cols = ["budget", "revenue", "vote_average", "popularity", "runtime"]
for c in numeric_cols:
    df_movies = df_movies.withColumn(c, col(c).cast("double"))

# Extract release year safely
df_movies = df_movies.withColumn("release_year", year(to_date(col("release_date"), "yyyy-MM-dd")))

# Count genres as a simple numeric feature
df_movies = df_movies.withColumn("genre_count", size(split(col("genres"), "\\|")))

# Fill nulls in numeric columns
df_movies = df_movies.fillna({c:0 for c in numeric_cols})


In [0]:
#Clean movies data

from pyspark.sql.functions import expr, col, try_to_date, year, size, split

# List of numeric columns
numeric_cols = ["budget", "revenue", "vote_average", "popularity", "runtime"]

# Safely cast to double using try_cast()
for c in numeric_cols:
    df_movies = df_movies.withColumn(c, expr(f"try_cast({c} as double)"))

# Safely extract release_year using try_to_date
df_movies = df_movies.withColumn("release_year", year(try_to_date(col("release_date"), "yyyy-MM-dd")))

# Count number of genres
df_movies = df_movies.withColumn("genre_count", size(split(col("genres"), "\\|")))

# Fill nulls in numeric columns
df_movies = df_movies.fillna({c: 0 for c in numeric_cols})



In [0]:
#Filter movies data
df_filtered = df_movies.filter(col("release_year").isNotNull())\
                       .filter(col("release_year") >= 2010)\
                       .filter(col("vote_average") > 7)

display(df_filtered.limit(5))


adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,genre_count
False,null,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",null,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring an imaginative young boy named Marek who dreams of escaping an increasingly dangerous Poland on the eve of war for beautiful Venice.,0.14713,/lUmJiBTKdesFDkgSvV9zecCgNO6.jpg,[],[],2010-05-25,0.0,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,null,Venice,False,7.5,4,2010,1
False,null,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'name': 'Horror'}]",null,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one told the new kid. It's gonna be a long night.",0.135596,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,[],[],2013-10-12,0.0,6.0,[],Released,null,The Sleepover,False,8.0,1,2013,1
False,null,0.0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, THE FARMER'S WIFE stoically carries out her daily tasks in preparation for what will be her final day on her farm, the only land she has known. Unable to accept this forced future, subconscious memories return that ultimately lead her to connect more to the countryside of her youth than ever before.",0.211754,/ePPNVWyIKYBdsGrOrYVaPKM8DlM.jpg,[],[],2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,null,The Farmer's Wife,False,10.0,1,2012,1
False,null,0.0,"[{'id': 35, 'name': 'Comedy'}]",http://brokenkingdomfilm.com/#/kingdomcome,136558,tt2043893,en,Kingdom Come,"The documentary, Kingdom Come follows a first-time director (Daniel Gillies) as he tries to raise a million dollars to finance his first film, Broken Kingdom. This emotionally-charged journey is interwoven with over 30 rare interviews from acclaimed indie darlings including Mark Ruffalo, Illeana Douglas, Don Cheadle, Kevin Smith, Edward Burns, Tim Roth, Morgan Spurlock, Selma Blair, Robert Townsend, Bill Pullman and many more.",0.986251,/rIafz3Ez5gH2b3USkZ5ab8VSd4Y.jpg,[],[],2011-01-01,0.0,88.0,[],Released,The unmaking of independent film,Kingdom Come,False,8.0,1,2011,1
False,null,2000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}]",null,40660,tt1380130,en,Jos rakastat,Musiikkielokuva joka kertoo ministerin tyttären ja maahanmuuttajien pojan rosoisen rakkaustarinan.,0.368731,/xQGmeW2DzM4iW9HxORwmNhiy4Tf.jpg,[],"[{'iso_3166_1': 'FI', 'name': 'Finland'}]",2010-01-06,0.0,121.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Menneisyyttä ei voi paeta ikuisesti. Antaako rakkaus lopulta kaiken anteeksi?,If You Love,False,7.5,4,2010,1


In [0]:
#Clean credits and keywords for joins

# Keep only rows with valid IDs
df_credits_clean = df_credits.filter(col("id").isNotNull())
df_keywords_clean = df_keywords.filter(col("id").isNotNull())
df_movies_clean = df_movies.filter(col("id").isNotNull())

# Safely cast IDs to int
df_credits_clean = df_credits_clean.withColumn("id", expr("try_cast(id as int)"))
df_keywords_clean = df_keywords_clean.withColumn("id", expr("try_cast(id as int)"))
df_movies_clean = df_movies_clean.withColumn("id", expr("try_cast(id as int)"))


In [0]:
#JOINS
df_joined = df_movies_clean.join(df_credits_clean, on="id", how="left")\
                           .join(df_keywords_clean, on="id", how="left")

display(df_joined.limit(5))



id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,genre_count,cast,crew,keywords
198317,False,null,850000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'name': 'Music'}]",null,tt0121411,en,Jacques Brel Is Alive and Well and Living in Paris,"Three attendees at a puppet theater don various roles in order to sing a variety of songs by Jacques Brel, all while hippies and other eccentrics cavort about them.",2.1E-5,/3kiWXvxSo7DHBU14f2AX16VQZSD.jpg,"[{'name': 'The American Film Theatre', 'id': 20151}]","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",1975-01-27,0.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,null,Jacques Brel Is Alive and Well and Living in Paris,False,0.0,0,1975,1,"[{'cast_id': 0, 'character': 'Himself', 'credit_id': '5302bb91c3a3680a114cb8ec', 'gender': 0, 'id': 24377, 'name': 'Jacques Brel', 'order': 1, 'profile_path': '/sDK7ZlibbTqEqOgDATn7vip8TZo.jpg'}]","[{'credit_id': '5302bc94c3a3680a3f481fb1', 'department': 'Directing', 'gender': 2, 'id': 1086649, 'job': 'Director', 'name': 'Denis Heroux', 'profile_path': None}]","[{'id': 10181, 'name': 'based on play or musical'}, {'id': 188715, 'name': 'musical revue'}]"
51933,False,null,1000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}]",http://www.handsomeharrythemovie.com/,tt1318001,en,Handsome Harry,"An ex-Navy man carrying out the last wish of a dying shipmate renews contact with old friends to break the code of silence around a mysterious, long-buried crime.",0.834019,/solugFmCfz8k5Vo3cNJGafEGv7S.jpg,[],[],2009-04-25,0.0,94.0,[],Released,Can we ever change the past?,Handsome Harry,False,5.3,7,2009,1,"[{'cast_id': 1001, 'character': 'Harry Sweeney', 'credit_id': '52fe480dc3a36847f8155ea9', 'gender': 2, 'id': 2977, 'name': 'Jamey Sheridan', 'order': 0, 'profile_path': '/xOhl3hP49XKTZuj8hzs2CdtMliT.jpg'}, {'cast_id': 1002, 'character': 'Thomas Kelley', 'credit_id': '52fe480dc3a36847f8155ead', 'gender': 2, 'id': 884, 'name': 'Steve Buscemi', 'order': 1, 'profile_path': '/e19GfOWzMNN1hi7B9Ci62hMvtXs.jpg'}, {'cast_id': 1004, 'character': 'Prof. Porter', 'credit_id': '52fe480dc3a36847f8155eb5', 'gender': 2, 'id': 18992, 'name': 'Aidan Quinn', 'order': 2, 'profile_path': '/fOiuJIBJUdiuqjQtGkffpSbRQ49.jpg'}, {'cast_id': 1006, 'character': 'David Kagan', 'credit_id': '58c7fa51c3a3683d9500ca08', 'gender': 2, 'id': 55152, 'name': 'Campbell Scott', 'order': 3, 'profile_path': '/LpYIN2jh58sHaUFM2r3G4G0iE6.jpg'}, {'cast_id': 1005, 'character': 'Peter Rheems', 'credit_id': '58c7fa3cc3a3683ddf00f316', 'gender': 2, 'id': 47879, 'name': 'John Savage', 'order': 4, 'profile_path': '/4cNKdr2E2gAvAjjydcwJGscoBsD.jpg'}, {'cast_id': 1007, 'character': 'Gebhardt', 'credit_id': '58c7fa6fc3a3683dc500e0f6', 'gender': 2, 'id': 39389, 'name': 'Titus Welliver', 'order': 5, 'profile_path': '/4CPGRpTuy6naurhkvRgsuae1qKR.jpg'}, {'cast_id': 1008, 'character': 'Muriel', 'credit_id': '58c7fa81c3a3683ddf00f353', 'gender': 1, 'id': 16215, 'name': 'Karen Young', 'order': 6, 'profile_path': '/aDFDZj0zLHDiv0vV8N8CKg48A6q.jpg'}, {'cast_id': 1009, 'character': 'Mrs. Schroeder', 'credit_id': '58c7faafc3a3683df300f0c9', 'gender': 1, 'id': 80135, 'name': 'Rutanya Alda', 'order': 7, 'profile_path': '/gVOyv8nNYwXmQU7kixKxILUMM50.jpg'}, {'cast_id': 1003, 'character': 'Judy Rheems', 'credit_id': '52fe480dc3a36847f8155eb1', 'gender': 1, 'id': 145058, 'name': 'Mariann Mayberry', 'order': 8, 'profile_path': None}]","[{'credit_id': '52fe480dc3a36847f8155ea5', 'department': 'Directing', 'gender': 1, 'id': 145057, 'job': 'Director', 'name': 'Bette Gordon', 'profile_path': None}]","[{'id': 10183, 'name': 'independent film'}, {'id': 187056, 'name': 'woman director'}]"
38742,Fal

In [0]:
# Movie data by release year — counting how many movies were released, their average rating, average budget, and average revenue.
from pyspark.sql.functions import round

df_yearly_stats = (
    df_joined
    .filter(col("release_year").isNotNull())
    .groupBy("release_year")
    .agg(
        count("*").alias("movie_count"),
        round(avg("vote_average"), 2).alias("avg_rating"),
        round(avg("budget"), 0).alias("avg_budget"),
        round(avg("revenue"), 0).alias("avg_revenue")
    )
    .orderBy("release_year")
)

display(df_yearly_stats)


release_year,movie_count,avg_rating,avg_budget,avg_revenue
1874,1,6.0,0.0,0.0
1878,1,6.2,0.0,0.0
1883,1,5.4,0.0,0.0
1887,1,4.1,0.0,0.0
1888,2,5.15,0.0,0.0
1890,3,5.3,0.0,0.0
1891,6,4.23,0.0,0.0
1892,3,3.47,0.0,0.0
1893,1,5.8,0.0,0.0
1894,11,5.11,0.0,0.0


In [0]:
# Register DataFrames as SQL views
df_movies.createOrReplaceTempView("movies")
df_credits.createOrReplaceTempView("credits")
df_keywords.createOrReplaceTempView("keywords")
df_joined.createOrReplaceTempView("joined_movies")



In [0]:
#Top 10 Highest-Rated Movies Since 2015

query1 = """
SELECT title, release_year, vote_average, popularity
FROM joined_movies
WHERE release_year >= 2015
ORDER BY vote_average DESC, popularity DESC
LIMIT 10
"""
df_top_movies = spark.sql(query1)
df_top_movies.show(truncate=False)


+-------------------------------------+------------+------------+----------+
|title                                |release_year|vote_average|popularity|
+-------------------------------------+------------+------------+----------+
|LEGO DC Super Hero Girls: Brain Drain|2017        |10.0        |8.413734  |
|Tokyo Ghoul                          |2017        |10.0        |0.802191  |
|Stephen Lynch: Hello Kalamazoo       |2016        |10.0        |0.724499  |
|Long Strange Trip                    |2017        |10.0        |0.617655  |
|The Human Surge                      |2016        |10.0        |0.484825  |
|First Round Down                     |2017        |10.0        |0.422836  |
|Sum of Histories                     |2015        |10.0        |0.404432  |
|Sum of Histories                     |2015        |10.0        |0.404432  |
|Bazodee                              |2016        |10.0        |0.379968  |
|My Future Love                       |2016        |10.0        |0.371238  |

In [0]:
#Query 2: Average Rating per Release Year

query2 = """
SELECT release_year, 
       ROUND(AVG(vote_average), 2) AS avg_rating, 
       COUNT(*) AS num_movies
FROM joined_movies
WHERE release_year IS NOT NULL
GROUP BY release_year
ORDER BY release_year DESC
"""
df_yearly_stats = spark.sql(query2)
df_yearly_stats.show()


+------------+----------+----------+
|release_year|avg_rating|num_movies|
+------------+----------+----------+
|        2020|       0.0|         1|
|        2018|       0.0|         4|
|        2017|      5.88|       511|
|        2016|      5.83|      1587|
|        2015|      5.77|      1930|
|        2014|      5.64|      1946|
|        2013|      5.75|      1822|
|        2012|      5.66|      1660|
|        2011|      5.62|      1595|
|        2010|      5.72|      1401|
|        2009|      5.59|      1499|
|        2008|      5.57|      1392|
|        2007|      5.66|      1222|
|        2006|      5.58|      1173|
|        2005|      5.58|      1055|
|        2004|      5.68|       904|
|        2003|      5.64|       804|
|        2002|      5.55|       837|
|        2001|       5.5|       808|
|        2000|      5.42|       726|
+------------+----------+----------+
only showing top 20 rows


In [0]:
output_path = f"{full_volume_path}processed_movies_data/"

# Save both query outputs
df_top_movies.write.mode("overwrite").parquet(output_path + "top_movies")
df_yearly_stats.write.mode("overwrite").parquet(output_path + "yearly_stats")

print("Results written successfully to:", output_path)


Results written successfully to: /Volumes/workspace/default/movies_data/processed_movies_data/


In [0]:
display(dbutils.fs.ls(output_path))


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/movies_data/processed_movies_data/top_movies/,top_movies/,0,1762886483052
dbfs:/Volumes/workspace/default/movies_data/processed_movies_data/yearly_stats/,yearly_stats/,0,1762886483052


In [0]:
df_loaded = spark.read.parquet(output_path + "top_movies")
df_loaded.show(5)


+--------------------+------------+------------+----------+
|               title|release_year|vote_average|popularity|
+--------------------+------------+------------+----------+
|LEGO DC Super Her...|        2017|        10.0|  8.413734|
|         Tokyo Ghoul|        2017|        10.0|  0.802191|
|Stephen Lynch: He...|        2016|        10.0|  0.724499|
|   Long Strange Trip|        2017|        10.0|  0.617655|
|     The Human Surge|        2016|        10.0|  0.484825|
+--------------------+------------+------------+----------+
only showing top 5 rows


In [0]:
# Performance Analysis 
# Analyze execution plan for first SQL query
print(" Execution Plan: Top Rated Movies Query ")
df_top_movies.explain(extended=True)

# Analyze execution plan for second SQL query
print("\n Execution Plan: Average Rating by Year Query ")
df_yearly_stats.explain(extended=True)


 Execution Plan: Top Rated Movies Query 
== Parsed Logical Plan ==
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Sort ['vote_average DESC NULLS LAST, 'popularity DESC NULLS LAST], true
      +- 'Project ['title, 'release_year, 'vote_average, 'popularity]
         +- 'Filter ('release_year >= 2015)
            +- 'UnresolvedRelation [joined_movies], [], false

== Analyzed Logical Plan ==
title: string, release_year: int, vote_average: double, popularity: double
GlobalLimit 10
+- LocalLimit 10
   +- Sort [vote_average#11484 DESC NULLS LAST, popularity#11481 DESC NULLS LAST], true
      +- Project [title#11393, release_year#11477, vote_average#11484, popularity#11481]
         +- Filter (release_year#11477 >= 2015)
            +- SubqueryAlias joined_movies
               +- View (`joined_movies`, [id#11521, adult#11373, belongs_to_collection#11374, budget#11480, genres#11376, homepage#11377, imdb_id#11379, original_language#11380, original_title#11381, overview#11382, popularity#11481, poster

In [0]:
#Actions vs Transformations

from pyspark.sql.functions import col

# Demonstrate lazy (transformation) vs eager (action) evaluation

# Transformation example: this line defines a new DataFrame but doesn’t execute anything yet
df_lazy = df_filtered.filter(col("budget") > 10000000)

# No computation happens here — Spark builds the DAG but doesn’t run it
print("Transformation defined. No action executed yet.")

# Action example: triggers actual computation
count_result = df_lazy.count()

print(f"Eager Action Triggered: Count of high-budget movies = {count_result}")


Transformation defined. No action executed yet.
Eager Action Triggered: Count of high-budget movies = 172


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Select relevant columns
ml_df = df_filtered.select("budget", "popularity", "runtime", "vote_average", "revenue") \
                   .na.fill(0)

# Prepare feature vector
assembler = VectorAssembler(
    inputCols=["budget", "popularity", "runtime", "vote_average"],
    outputCol="features"
)

ml_df = assembler.transform(ml_df)

# Split into train/test sets
train_df, test_df = ml_df.randomSplit([0.8, 0.2], seed=42)

# Define a simple linear regression model
lr = LinearRegression(featuresCol="features", labelCol="revenue")

# Fit the model
lr_model = lr.fit(train_df)

# Evaluate on test data
predictions = lr_model.transform(test_df)

# Evaluate model performance
evaluator = RegressionEvaluator(
    labelCol="revenue",
    predictionCol="prediction",
    metricName="rmse"
)
rmse = evaluator.evaluate(predictions)

print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Model Coefficients: {lr_model.coefficients}")
print(f"Model Intercept: {lr_model.intercept}")

# Show sample predictions
predictions.select("budget", "popularity", "runtime", "vote_average", "revenue", "prediction").show(5)


Root Mean Squared Error (RMSE): 59135721.21
Model Coefficients: [4.479611175832705,581457.5632309138,-64651.92346010584,1943420.2199460126]
Model Intercept: -11375937.95455873
+------+----------+-------+------------+-------+-------------------+
|budget|popularity|runtime|vote_average|revenue|         prediction|
+------+----------+-------+------------+-------+-------------------+
|   0.0|  0.211754|   18.0|        10.0|    0.0|  7017655.587463889|
|   0.0|  0.109599|  255.0|        10.0|    0.0| -8364249.069953049|
|   0.0|  0.616048|  118.0|         7.3|    0.0|  -4459691.54833205|
|   0.0|  2.636798|  125.0|         7.6|    0.0|-3154248.5756701184|
|   0.0|  2.775816|  102.0|         7.3|    0.0| -2169447.334546253|
+------+----------+-------+------------+-------+-------------------+
only showing top 5 rows
